# Manual Data Augmentation

In [ ]:
import glob
import os
from PIL import Image
import numpy as np

"""
PIL reads image to array [0, 255], int type.
perform four types of image transformation
"""
class ImageGenerator():
    def __init__(self, source_dir, target_dir, enlarge_size=(171, 171), crop_size=(150, 150), channel=3):
        self.source_dir = source_dir
        self.target_dir = target_dir
        self.enlarge_size = enlarge_size
        self.crop_size = crop_size
        self.PCA = {}
        self.channel = channel        
        
    def left_right_flip(self, image_arr):
        return np.fliplr(image_arr)

    def top_bottom_flip(self, image_arr):
        return np.flipud(image_arr)
    
    def enlarge_and_crop(self, image):
        im_enlarge = image.resize(self.enlarge_size)
        im_enlarge_arr = np.array(im_enlarge)
        e1, e2 = self.enlarge_size
        c1, c2 = self.crop_size
        x_lim = e1 - c1
        y_lim = e2 - c2
        x = int(np.random.uniform(0, x_lim))
        y = int(np.random.uniform(0, y_lim))
        im_crop_arr = im_enlarge_arr[x:x + c1, y:y+c2, :]
        return im_crop_arr
    
    def PCA_RGB(self):
        # find PCA component of RGB channels throughout whole dataset
        image_paths = glob.glob(os.path.join(self.source_dir, '*.png'))
        
        im_arrs = []
        
        for path in image_paths:
            im_arr = np.array(Image.open(path))
            im_arr = im_arr.reshape(-1, self.channel)
            im_arrs.append(im_arr)
            
        arr = np.concatenate(im_arrs)
        im_cov = np.cov(arr.T)
        U, s, Vt = np.linalg.svd(im_cov)
        self.PCA['U'] = U
        self.PCA['lambda'] = np.sqrt(s)
            
    def PCA_color_augmentation(self, image_arr):
        H, L, C = image_arr.shape
        mu, sigma = 0, 0.1
        vec_alpha = np.random.normal(mu, sigma, 3)
        im_color_shifted = image_arr.reshape(H*L, C) + np.dot(self.PCA['U'], (vec_alpha * self.PCA['lambda']).T)
        im_color_shifted = np.clip(im_color_shifted, 0, 255)
        im_color_shifted = im_color_shifted.reshape(H, L, C)
        return im_color_shifted.astype('uint8')   
    
    def process(self):
        image_paths = glob.glob(os.path.join(self.source_dir, '*.png'))
        
        self.PCA_RGB()
        
        for path in image_paths:
            image = Image.open(path)
            image_arr = np.array(image)
            fname = os.path.basename(path).split(".png")[0]
            
            # save image of left right flip
            Image.fromarray(self.left_right_flip(image_arr)).save(os.path.join(target_dir, fname+"_lr.png"))
            # save image of top bottom flip
            Image.fromarray(self.top_bottom_flip(image_arr)).save(os.path.join(target_dir, fname+"_tb.png"))
            # save image of random crop
            Image.fromarray(self.enlarge_and_crop(image)).save(os.path.join(target_dir, fname+"_rc.png"))
            # save image of PCA color augmentation
            Image.fromarray(self.PCA_color_augmentation(image_arr)).save(os.path.join(target_dir, fname+"_pca.png"))
            

In [ ]:
# Data augmentation on training set
source_dir = ".\\data\\train"
target_dir = ".\\data\\train"
dataGen_train = ImageGenerator(source_dir, target_dir)
dataGen_train.process()